In [29]:
import numpy as np
import pandas as pd
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.244:7077") \
        .appName("Group10_DE")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

sc = spark_session.sparkContext

sc.setLogLevel("ERROR")


In [30]:
df = spark_session.read.json("hdfs://host-192-168-2-244-de1:9000/user/ubuntu/corpus-webis-tldr-17.json")


In [31]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [37]:
df.count()

3848330

In [36]:
pp_df = df.na.drop('any')
pp_df.count()

1762893

In [38]:

res = pp_df.select('subreddit', 'subreddit_id', 'content_len') \
  .groupBy('subreddit_id', 'subreddit') \
  .agg(F.count('subreddit_id').alias('subreddit_count'),
       F.avg('content_len').alias('average_content_len')) \
  .orderBy(F.desc('subreddit_count'))

res.show(10)

+------------+-------------------+---------------+-------------------+
|subreddit_id|          subreddit|subreddit_count|average_content_len|
+------------+-------------------+---------------+-------------------+
|    t5_2qjvn|      relationships|         348493|  462.2536521537018|
|    t5_2qh1i|          AskReddit|         100498| 269.38906246890485|
|    t5_2rfxx|    leagueoflegends|          61912|    246.47417301977|
|    t5_2to41|               tifu|          50243|  352.5128674641244|
|    t5_2r0cn|relationship_advice|          46339| 472.32674421113967|
|    t5_2r9vp|              trees|          31334| 261.78971724005874|
|    t5_2qh3p|                sex|          18862|  328.2478528257873|
|    t5_2qh2p|            atheism|          15167|  331.3584097052812|
|    t5_2ranw|         offmychest|          15124|  609.6938640571277|
|    t5_2vq0w|     DestinyTheGame|          14313|  314.8230978830434|
+------------+-------------------+---------------+-------------------+
only s

In [40]:
res.agg(F.avg('average_content_len')).show()

+------------------------+
|avg(average_content_len)|
+------------------------+
|      307.49167245323974|
+------------------------+



In [41]:
sc.stop()